In [1]:
import numpy as np
import os
from sklearn.metrics import accuracy_score
from sktime.utils.load_data import load_from_tsfile_to_dataframe
from tqdm.notebook import tqdm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.cluster import KMeans

# from tslearn.neighbors import KNeighborsTimeSeriesClassifier

import logging
# logging.basicConfig(filename="1_nn_eucledian_results_history.log", level=logging.INFO)

import warnings
warnings.filterwarnings("ignore")

## Loading and preprocessing

In [2]:
def simple_preproc_before_knn(X):
    X = X.applymap(np.array)
    dims_lst = X.columns

    for dim_name in dims_lst:
        dim_values = np.stack(X[dim_name].values, axis=0)
        for i in range(dim_values.shape[1]):
            X[dim_name + f'_{i}'] = dim_values[:, i]

    X = X.drop(columns=dims_lst)
    return X


def preproc_answers(y):
    le = preprocessing.LabelEncoder()
    y = le.fit_transform(y)
    return y

In [3]:
datasets_directory = "/root/data/Multivariate_ts/"

# dataset_name = 'PenDigits'
dataset_name = 'LSST'


X_train, y_train = load_from_tsfile_to_dataframe(datasets_directory + dataset_name + f'/{dataset_name}_TRAIN.ts')
X_test, y_test = load_from_tsfile_to_dataframe(datasets_directory + dataset_name + f'/{dataset_name}_TEST.ts')

X_train = simple_preproc_before_knn(X_train)
X_test = simple_preproc_before_knn(X_test)

ms = preprocessing.MinMaxScaler()
X_train = ms.fit_transform(X_train)
X_test = ms.transform(X_test)

y_train = preproc_answers(y_train)
y_test = preproc_answers(y_test)

## Clusterization

In [21]:
classes_num = len(np.unique(y_train))
km = KMeans(n_clusters=classes_num, random_state=11)

km.fit(X_train)
train_labels = km.labels_

km.fit(X_test)
test_labels = km.labels_

## Classification with new labels

In [22]:
knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn.fit(X_train, train_labels)

# train_predictions = knn.predict(X_train)
# train_accuracy = accuracy_score(train_predictions, train_labels)
# print('train_accuracy:', round(train_accuracy,5))

test_predictions = knn.predict(X_test)
test_accuracy = accuracy_score(test_predictions, test_labels)
print('test_accuracy:', round(test_accuracy,5))

test_accuracy: 0.00365
